## Data reading and manipulation

In [1]:
# activate the environment
using Pkg
Pkg.activate("/home/eneserciyes/projects/WoR/world_on_rails.jl")

  Activating environment at `~/projects/WoR/world_on_rails.jl/Project.toml`


In [3]:
using YAML: load_file
using Images
using Images.FileIO
using StrLiterals
using LMDB
using LMDB: create, open, close, put!, start

In [2]:
DATA_ROOT = "/home/eneserciyes/projects/WoR/dataset"
CONFIG_PATH = "/home/eneserciyes/projects/WoR/world_on_rails.jl/configs/config.yaml"

"/home/eneserciyes/projects/WoR/world_on_rails.jl/configs/config.yaml"

In [15]:
struct MainDataset
    config::Dict
    num_frames::Int
    txn_map::Dict
    idx_map::Dict
    yaw_map::Dict
    file_map::Dict

    function MainDataset(data_dir::AbstractString, config_path::AbstractString)
        # read the config file
        num_frames = 0
        config = load_file(config_path)
        txn_map = Dict()
        idx_map = Dict()
        yaw_map = Dict()
        file_map = Dict()
        
        for full_path in readdir(data_dir, join=true)
            if !isdir(full_path)
                continue
            end
            env = create()
            open(env, full_path)
            txn = start(env)

            dbi = open(txn)
            
            n = parse(Int64, LMDB.get(txn, dbi, "len", String))
            if n < config["num_plan"]+1
                print(full_path, " is too small. consider deleting")
                close(env, dbi)
            else
                offset = num_frames
                for i in 0:n-config["num_plan"]-1
                    num_frames+=1
                    for j in 0:length(config["camera_yaws"])-1
                        txn_map[(offset+i)*length(config["camera_yaws"])+j] = txn
                        idx_map[(offset+i)*length(config["camera_yaws"])+j] = i
                        yaw_map[(offset+i)*length(config["camera_yaws"])+j] = j
                        file_map[(offset+i)*length(config["camera_yaws"])+j] = full_path
                    end
                end
            end
        end
        print(data_dir,": ", num_frames, " frames ", "x ", length(config["camera_yaws"]))
        new(config, num_frames, txn_map, idx_map, yaw_map, file_map)
    end
end

In [16]:
main_dataset = MainDataset(DATA_ROOT, CONFIG_PATH)

/home/eneserciyes/projects/WoR/dataset: 195 frames x 3

MainDataset(Dict{Any, Any}("ego_model_dir" => "/kuacc/users/merciyes18/WorldOnRails/ego_model.th", "num_plan" => 5, "camera_yaws" => [0, -30, 30], "multi_cam" => true, "num_orient" => 5, "num_throts" => 3, "all_speeds" => true, "temperature" => 0.01, "ego_data_dir" => ["PATH TO PHASE 0 DATA"], "min_speeds" => 0.0…), 195, Dict{Any, Any}(319 => Transaction(Ptr{Nothing} @0x0000000003772620), 185 => Transaction(Ptr{Nothing} @0x0000000003772620), 420 => Transaction(Ptr{Nothing} @0x0000000003772620), 525 => Transaction(Ptr{Nothing} @0x0000000003772620), 365 => Transaction(Ptr{Nothing} @0x0000000003772620), 263 => Transaction(Ptr{Nothing} @0x0000000003772620), 422 => Transaction(Ptr{Nothing} @0x0000000003772620), 242 => Transaction(Ptr{Nothing} @0x0000000003772620), 183 => Transaction(Ptr{Nothing} @0x0000000003772620), 551 => Transaction(Ptr{Nothing} @0x0000000003772620)…), Dict{Any, Any}(319 => 106, 185 => 61, 420 => 140, 525 => 175, 365 => 121, 263 => 87, 422 => 140, 242 => 80, 183 => 61, 5

In [6]:
Base.length(d::MainDataset) = (if (d.config["multi_cam"]) d.num_frames*length(d.config["camera_yaws"]) else d.num_frames end)

In [7]:
length(main_dataset)

585

In [26]:
function get_item(d::MainDataset, state::Int)
    T = d.config["num_plan"]
    idx = state
    if !d.config["multi_cam"]
        idx *= length(d.config["camera_yaws"])
    end
    
    lmdb_txn = d.txn_map[idx]
    index = d.idx_map[idx]
    cam_index = d.yaw_map[idx]
    
    locs = access("loc", lmdb_txn, index, T+1, Vector{Float32})
    locs
end

get_item (generic function with 1 method)

In [37]:
Base.iterate(d::MainDataset, state=1) = state > length(d) ? nothing : get_item(d, state), state+1 

In [1]:
get_item(main_dataset, 1)

LoadError: UndefVarError: get_item not defined